# L5c: A Deeper Dive into Flux Balance Analysis Constraints
In this lecture, we will continue our discussion of flux balance analysis and explore what we can do with the material balance and flux bounds constraints. However, before we do that, let's review the key concepts we'll discuss today.

* __Flux balance analysis (FBA)__ is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where metabolite production, consumption, and transport rates are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. 
* __Material balance constraints__ are used to ensure that the flow of metabolites into and out of a system is balanced (and physical). These constraints are typically represented as a set of linear equations, where the coefficients represent the stoichiometry of the reactions in the system in combination with transport and dilution terms.
* __Flux bounds constraints__ limit the range of possible fluxes through a metabolic network. These bounds can incorporate additional information, such as experimental data or prior knowledge about the system, into the FBA problem.

Lecture notes can be downloaded: [here!](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5c/docs/Notes.pdf)

## Flux Balance Analysis Problem Formulation
Flux balance analysis (FBA) is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where metabolite production, consumption, and transport rates are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. The linear program is:
$$
\begin{align*}
\max_{\hat{v}}\quad&  \sum_{i\in\mathcal{R}}c_{i}\hat{v}_{i}\\
\text{subject to}\quad & \sum_{s\in\mathcal{S}}d_{s}C_{i,s}\dot{V}_{s} + \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}V = \frac{d}{dt}\left(C_{i}V\right)\qquad\forall{i\in\mathcal{M}}\\
& \mathcal{L}_{j}\leq\hat{v}_{j}\leq\mathcal{U}_{j}\qquad\forall{j\in\mathcal{R}}
\end{align*}
$$
Here, $\sigma_{ij}$ are elements of the stoichiometric matrix $\mathbf{S}$, $c_{i}$ are objective coefficients (you choose), $\hat{v}_{i}$ are unknown fluxes, and $\mathcal{L}_{j}$ and $\mathcal{U}_{j}$ are the lower and upper flux bounds, respectively.

### Simplified Species Constraints
In the [FBA primer](https://pubmed.ncbi.nlm.nih.gov/20212490/) the material balance constraints were written as $\mathbf{S}\hat{\mathbf{v}} = 0$. This is a simplification; in reality, the material balance constraints are more complex. But let's see how we get there.
When doing FBA, we often will make three assumptions:
* __Steady-state, constant volume__: The biological system (or at least part of it) is in a steady state, and in whole-cell models, the volume of the culture is constant. However, this is not the case in fed-batch cultures; many industrial biotechnology processes operate in fed-batch mode. 
* __Specific units__: The volume basis for the _intracellular_ species concentrations is in specific units, i.e., per unit cell mass measured in grams dry weight (units: `gDW`). For cell-free systems, the volume basis is the volume of the reactor.
* __No transport or dilution terms__: We will assume that there are no physical transport or dilution terms in the material balance equations. This is a simplification, but it is often used in FBA. For example, this assumption is not the case for continuous cell-free systems.

#### Palsson constraints
Let the volume of our system be written in specific units, i.e., $V=B\bar{V}$, where $B$ is the biomass concentration (units: `gDW/L`) and $\bar{V}$ is the volume of the culture (units: `L`). The material balance constraints can be simplified by assuming the species in our system are in a steady state. But there is more to this story. Let's expand the accumulation terms:
$$
\begin{align*}
\frac{d}{dt}\left(C_{i}V\right) &= \frac{d}{dt}\left(C_{i}B\bar{V}\right)\\
&= B\bar{V}\underbrace{\left(\frac{dC_{i}}{dt}\right)}_{\text{steady state}\,=\,0} + C_{i}B\underbrace{\left(\frac{d\bar{V}}{dt}\right)}_{\text{steady state\,=\,0}} + C_{i}\bar{V}\left(\frac{dB}{dt}\right)\\
C_{i}\bar{V}\left(\frac{dB}{dt}\right) & = \underbrace{\sum_{s\in\mathcal{S}}d_{s}C_{i,s}\dot{V}_{s}}_{\text{no transport\,=\,0}} + \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}V\\
C_{i}\bar{V}\left(\frac{dB}{dt}\right) & = \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}B\bar{V}\\
C_{i}\underbrace{\left[\frac{1}{B}\left(\frac{dB}{dt}\right)\right]}_{\text{specific growth rate $\mu$}} & = \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}\\
C_{i}\mu & = \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}\\
\sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j} - \underbrace{C_{i}\mu}_{\text{small}\,\ll{1}} & = 0\\
\sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j} & = 0\quad\forall{i\in\mathcal{M}}\quad\blacksquare
\end{align*}
$$

#### Exchange reactions
Great! So then, does everything in our system have to be at a steady state? Not exactly.  
* We can think of the system we are studying as an open system, i.e., it can exchange material with the surroundings. Thus, while the components of the system are in a steady state, the universe (system + surroundings) as a whole is not.
* This is a subtle point, but it is essential to understand. The exchange of material with the surroundings is captured in the context of our three assumptions by writing _hypothetical reactions_ that exchange material with the surroundings. We call these reactions [the _exchange reactions_](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5c/docs/figs/Fig-ExchangeReactions.png); this figure was reproduced from [Bordbar et al, 2014](https://pubmed.ncbi.nlm.nih.gov/24987116/).

## A model for flux bounds
The flux bounds are important constraints in flux balance analysis calculations and the convex decomposition of the stoichiometric array. Beyond their role in the flux estimation problem, the flux bounds are _integrative_, i.e., these constraints integrate many types of genetic and biochemical information into the problem. A general model for these bounds is given by:
$$
\begin{align*}
-\delta_{j}\underbrace{\left[{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}\right]}_{\text{reverse: other functions or parameters?}}\leq\hat{v}_{j}\leq{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}
\end{align*}
$$
where $V_{max,j}^{\circ}$ denotes the maximum reaction velocity (units: `flux`) computed at some _characteristic enzyme abundance_. Thus, the maximum reaction velocity is given by:
$$
V_{max,j}^{\circ} = k_{cat,j}^{\circ}e^{\circ}
$$
where $k_{cat,j}$ is the catalytic constant or turnover number for the enzyme (units: `1/time`) and $e^{\circ}$ is a characteristic enzyme abundance (units: `concentration`). The term $\left(e/e^{\circ}\right)$ is a correction to account for the _actual_ enzyme abundance catalyzing the reaction (units: `dimensionless`). The $\theta_{j}\left(\dots\right)\in\left[0,1\right]$ is the current fraction of maximial enzyme activity of enzyme $e$ in reaction $j$. The activity model $\theta_{j}\left(\dots\right)$ describes [allosteric effects](https://en.wikipedia.org/wiki/Allosteric_regulation) on the reaction rate, and is a function of the regulatory and the chemical state of the system, the concentration of substrates, products, and cofactors (units: `dimensionless`).
Finally, the $f_{j}\left(\dots\right)$ is a function describing the substrate (reactants) dependence of the reaction rate $j$ (units: `dimensionless`). 

* __Parameters__: We need estimates for the $k_{cat,j}^{\circ}$ for all enzymes in the system we are interested in and a _reasonable policy_ for specifying a characteristic value for $e^{\circ}$. In addition, the $\theta_{j}\left(\dots\right)$ and $f_{j}\left(\dots\right)$ models can also have associated parameters, e.g., saturation or binding constants, etc. Thus, we need to estimate these from literature studies or experimental data.
* __Reversibility__: Next, we need to estimate the binary direction parameter $\delta_{j}\in\left\{0,1\right\}$. The value of $\delta_{j}$ describes the reversibility of reaction $j$; if reaction $j$ is __reversible__ $\delta_{j}=1$. If reaction $j$ is __irreversible__ $\delta_{j}=0$

### Simplfied bounds model
Let's initially assume that $(e/e^{\circ})\sim{1}$, there are no allosteric inputs $\theta_{j}\left(\dots\right)\sim{1}$, and the substrates are saturating $f_{j}\left(\dots\right)\sim{1}$. 
Then, the flux bounds are given by:
$$
\begin{align*}
-\delta_{j}V_{max,j}^{\circ}\leq{\hat{v}_{j}}\leq{V_{max,j}^{\circ}}
\end{align*}
$$
This is a simple model for the flux bounds. It is easy to see that the flux bounds are a function of the maximum reaction velocity, the catalytic constant or turnover number, and our assumed value of a characteristic enzyme abundance.

### Turnover numbers
The turnover number, $k_{cat}$, measures an enzyme's catalytic activity, defined as the number of substrate molecules converted to product per enzyme molecule per unit time. Units are typically `1/time` (e.g., `1/s` or `1/min`). Values can be obtained from primary literature or databases like [BRENDA](https://www.brenda-enzymes.org/):

* [Antje Chang et al., BRENDA, the ELIXIR core data resource in 2021: new developments and updates, Nucleic Acids Research, Volume 49, Issue D1, 8 January 2021, Pages D498–D508, https://doi.org/10.1093/nar/gkaa1025](https://academic.oup.com/nar/article/49/D1/D498/5992283)

Use [BRENDA](https://www.brenda-enzymes.org/) to find turnover numbers for:
* __Enzyme 1__: Arginase (EC 3.5.3.1) in humans.
* __Enzyme 2__: Argininosuccinate synthase (EC 6.3.4.5) in humans.

### eQuilibrator
The second thing we need to estimate is the reversibility parameter.
The reversibility parameter $\delta_{j}$ can be computed in several ways. For example, [one method in the literature](https://pubmed.ncbi.nlm.nih.gov/27159581/) is to use the sign of Gibbs reaction energy:
$$
\begin{equation*}
\delta_{i} = \begin{cases}
0 & \text{if }\text{sign}\left(\Delta{G}^{\circ} - \Delta{G}^{\star}\right)= -1 \quad\text{irreversible} \\
1 & \text{if }\text{sign}\left(\Delta{G}^{\circ} - \Delta{G}^{\star}\right)= +1 \quad\text{reversible}
\end{cases}
\end{equation*}
$$
where $\Delta{G}^{\circ}$ is the [standard Gibbs free energy change of the reaction](https://en.wikipedia.org/wiki/Gibbs_free_energy#Gibbs_free_energy_of_reactions), and $\Delta{G}^{\star}$ is a threshold value (hyperparameter). The threshold value can be set to zero or some other value. Alternatively, the value of $\delta_{j}$ can be assigned based upon a cutoff $K^{\star}$ on the equilibrium constant:
$$
\begin{equation*}
\delta_{i} = \begin{cases}
0 & \text{if }K_{eq}>\,K^{\star}\quad\text{irreversible} \\
1 & \text{if }K_{eq}\leq\,K^{\star}\quad\text{reversible}
\end{cases}
\end{equation*}
$$
where you specify the value $K^{\star}$ based upon some intution or other criteria. We can compute the $\Delta{G}^{\circ}$ values using [eQuilibrator](https://equilibrator.weizmann.ac.il):
* [Beber ME, Gollub MG, Mozaffari D, Shebek KM, Flamholz AI, Milo R, Noor E. eQuilibrator 3.0: a database solution for thermodynamic constant estimation. Nucleic Acids Res. 2022 Jan 7;50(D1): D603-D609. doi: 10.1093/nar/gkab1106. PMID: 34850162; PMCID: PMC8728285.](https://pubmed.ncbi.nlm.nih.gov/34850162/)
The [eQuilibrator application programming interface](https://equilibrator.weizmann.ac.il) is a tool for thermodynamic calculations in biological reaction networks. It was developed by the [Milo lab](https://www.weizmann.ac.il/plants/Milo/) at the Weizmann Institute in Rehovot, Israel. The [`eQuilibrator.jl` package](https://github.com/stelmo/eQuilibrator.jl) is a [Julia](https://julialang.org) wrapper around eQuilibrator (which is written in Python). 

Use [eQuilibrator](https://equilibrator.weizmann.ac.il) to find the $\delta$ values for:
* __Enzyme 1__: Arginase (EC 3.5.3.1) in humans.
* __Enzyme 2__: Argininosuccinate synthase (EC 6.3.4.5) in humans

where we assume a threshold value of $\Delta{G}^{\star}=-5.5$ kJ/mol (__hmmm__: where did this come from)?

## PS3 (Preview): Flux Balance Analysis of the Urea Cycle in HL-60 Cells
The [urea cycle](https://www.kegg.jp/pathway/hsa00220) is a crucial metabolic pathway that converts toxic ammonia into urea for excretion. While the urea cycle's role in [HL-60 cells, a human promyelocytic leukemia cell line](https://www.atcc.org/products/ccl-240?matchtype=b&network=g&device=c&adposition=&keyword=hl60%20cell%20line%20atcc&gad_source=1&gbraid=0AAAAADR6fpoOXsp8U8fXLd_E6sLTcwv24&gclid=CjwKCAiA5eC9BhAuEiwA3CKwQm0C1oE5_JjTpJ24VnTjZUZQVLivpPxmufDo7HdH5v3hN1XKnEf3ExoCvhwQAvD_BwE), is not directly established, these cells exhibit alterations in protein levels and proliferation rates when exposed to various compounds, which may indirectly affect nitrogen metabolism and related pathways.

In problem set 3, we will explore the urea cycle in HL-60 cells using flux balance analysis. We'll construct [a simplified model of the urea cycle](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5c/docs/figs/Fig-Urea-cycle-Schematic.pdf), analyze its structure, determing reversibility and the bounds, and then compute the flux distribution through the network under different assumptions.

### References
1. [Al-Otaibi NAS, Cassoli JS, Martins-de-Souza D, Slater NKH, Rahmoune H. Human leukemia cells (HL-60) proteomic and biological signatures underpinning cryo-damage are differentially modulated by novel cryo-additives. Gigascience. 2019 Mar 1;8(3):giy155. doi: 10.1093/gigascience/giy155. PMID: 30535373; PMCID: PMC6394207.](https://pmc.ncbi.nlm.nih.gov/articles/PMC6394207/)
2. [Figarola JL, Weng Y, Lincoln C, Horne D, Rahbar S. Novel dichlorophenyl urea compounds inhibit proliferation of human leukemia HL-60 cells by inducing cell cycle arrest, differentiation and apoptosis. Invest New Drugs. 2012 Aug;30(4):1413-25. doi: 10.1007/s10637-011-9711-8. Epub 2011 Jul 5. PMID: 21728022.](https://pubmed.ncbi.nlm.nih.gov/21728022/)
3. [Caldwell RW, Rodriguez PC, Toque HA, Narayanan SP, Caldwell RB. Arginase: A Multifaceted Enzyme Important in Health and Disease. Physiol Rev. 2018 Apr 1;98(2):641-665. doi: 10.1152/physrev.00037.2016. PMID: 29412048; PMCID: PMC5966718.](https://pmc.ncbi.nlm.nih.gov/articles/PMC5966718/)

# Today?
That's a wrap! What are some things we discussed today?